### Entrenamiento de un modelo SVM implementado con HOG

In [397]:
import os
import cv2 as cv
import numpy as np
import pickle
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Definir rutas para las imágenes de entrenamiento
base_path = r'C:\Users\Royer\Downloads\onta_waldo\onta_waldo\Dataset_waldo'
waldo_dir = os.path.join(base_path, 'waldo')
no_waldo_dir = os.path.join(base_path, 'no_waldo')

# Función para extraer características HOG de una imagen
def obtener_caracteristicas(imagen, tamaño=(64, 128)):
    # Convertir la imagen a escala de grises
    gris = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
    # Redimensionar la imagen
    redimensionada = cv.resize(gris, tamaño)

    # Definir el descriptor HOG
    hog_descriptor = cv.HOGDescriptor(_winSize=(tamaño[0] // 8 * 8, tamaño[1] // 8 * 8),
                                      _blockSize=(16, 16),
                                      _blockStride=(8, 8),
                                      _cellSize=(8, 8),
                                      _nbins=9)
    # Computar el descriptor HOG
    hog_caracteristicas = hog_descriptor.compute(redimensionada)
    return hog_caracteristicas.flatten()

# Listas para almacenar las características y las etiquetas
caracteristicas_data = []
etiquetas_data = []

# Cargar y procesar las imágenes positivas (con Wally)
for archivo_img in os.listdir(waldo_dir):
    ruta_img = os.path.join(waldo_dir, archivo_img)
    imagen = cv.imread(ruta_img)
    if imagen is not None:
        caracteristicas = obtener_caracteristicas(imagen)
        caracteristicas_data.append(caracteristicas)
        etiquetas_data.append(1)  # Etiqueta 1 para imágenes con Wally

# Cargar y procesar las imágenes negativas (sin Wally)
for archivo_img in os.listdir(no_waldo_dir):
    ruta_img = os.path.join(no_waldo_dir, archivo_img)
    imagen = cv.imread(ruta_img)
    if imagen is not None:
        caracteristicas = obtener_caracteristicas(imagen)
        caracteristicas_data.append(caracteristicas)
        etiquetas_data.append(0)  # Etiqueta 0 para imágenes sin Wally

# Convertir las listas a arrays de NumPy
caracteristicas_data = np.array(caracteristicas_data)
etiquetas_data = np.array(etiquetas_data)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(caracteristicas_data, etiquetas_data, test_size=0.5, random_state=80)

# Crear y entrenar el modelo SVM con kernel lineal
modelo_svm = svm.SVC(kernel='linear', probability=True)
modelo_svm.fit(X_train, y_train)

# Predecir las etiquetas del conjunto de prueba
y_pred = modelo_svm.predict(X_test)

# Calcular y mostrar la precisión del modelo
precision = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {precision * 100:.2f}%')

# Guardar el modelo entrenado en un archivo
with open('modelo_svm_donde_esta_wally.pkl', 'wb') as modelo_file:
    pickle.dump(modelo_svm, modelo_file)
print("Modelo SVM guardado como 'modelo_svm_donde_esta_wally.pkl'")


Precisión del modelo: 85.49%
Modelo SVM guardado como 'wally_svm_model2.pkl'


### Busqueda de wally usando el modelo SVM

In [422]:
import cv2 as cv
import numpy as np
import pickle
import os

# Cargar el modelo SVM desde el archivo
modelo_svm_path = 'wally_svm_model2.pkl'
with open(modelo_svm_path, 'rb') as modelo_file:
    modelo_svm = pickle.load(modelo_file)

# Función para extraer características HOG de una imagen
def extraer_caracteristicas(imagen, tamaño=(64, 128)):
    imagen_gris = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
    imagen_redimensionada = cv.resize(imagen_gris, tamaño)
    descriptor_hog = cv.HOGDescriptor()
    caracteristicas = descriptor_hog.compute(imagen_redimensionada)
    return caracteristicas.flatten()

# Leer la imagen de entrada
ruta_imagen = 'C:\\Users\\Royer\\Downloads\\onta_waldo\\onta_waldo\\ejemplos\\4.png'
imagen = cv.imread(ruta_imagen)

# Ajustes de la ventana de detección
tamaño_ventana = (138, 138)
paso_ventana = 63

detec_encontradas = []
wally_encontrado = False

# Recorrer la imagen con una ventana deslizante
for y in range(0, imagen.shape[0] - tamaño_ventana[1], paso_ventana):
    for x in range(0, imagen.shape[1] - tamaño_ventana[0], paso_ventana):
        sub_imagen = imagen[y:y + tamaño_ventana[1], x:x + tamaño_ventana[0]]
        caracteristicas = extraer_caracteristicas(sub_imagen)
        prediccion = modelo_svm.predict([caracteristicas])
        if prediccion == 1:
            detec_encontradas.append((x, y, tamaño_ventana[0], tamaño_ventana[1]))

# Función para aplicar la supresión no máxima (NMS)
def supresion_no_maxima(cajas, umbral_solapamiento):
    if len(cajas) == 0:
        return []

    cajas = np.array(cajas)
    seleccionadas = []

    x1 = cajas[:, 0]
    y1 = cajas[:, 1]
    x2 = cajas[:, 0] + cajas[:, 2]
    y2 = cajas[:, 1] + cajas[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    indices = np.argsort(y2)

    while len(indices) > 0:
        ultimo = len(indices) - 1
        i = indices[ultimo]
        seleccionadas.append(i)

        xx1 = np.maximum(x1[i], x1[indices[:ultimo]])
        yy1 = np.maximum(y1[i], y1[indices[:ultimo]])
        xx2 = np.minimum(x2[i], x2[indices[:ultimo]])
        yy2 = np.minimum(y2[i], y2[indices[:ultimo]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        solapamiento = (w * h) / areas[indices[:ultimo]]

        indices = np.delete(indices, np.concatenate(([ultimo], np.where(solapamiento > umbral_solapamiento)[0])))

    return cajas[seleccionadas].astype("int")

# Aplicar la supresión no máxima a las detecciones
umbral_solapamiento = 0.1
detec_finales = supresion_no_maxima(detec_encontradas, umbral_solapamiento)

# Dibujar rectángulos alrededor de las detecciones
for (x, y, w, h) in detec_finales:
    cv.rectangle(imagen, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv.putText(imagen, 'Waldo', (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    wally_encontrado = True

# Imprimir mensaje si no se encontró a Wally
if not wally_encontrado:
    print("IDK.")

# Ajustar el tamaño de la ventana de visualización
altura, anchura, _ = imagen.shape
factor_redimension = 0.6
nueva_anchura = int(anchura * factor_redimension)
nueva_altura = int(altura * factor_redimension)

# Mostrar la imagen con las detecciones
cv.namedWindow('Donde esta Waldo', cv.WINDOW_NORMAL)
cv.resizeWindow('Donde esta Waldo', nueva_anchura, nueva_altura)
cv.imshow('Donde esta Waldo', imagen)
cv.waitKey(0)
cv.destroyAllWindows()
